<a href="https://colab.research.google.com/github/ndubuisijosephalx/Data-Science-projects/blob/main/nlp_Ndubuisi_J_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!jupyter contrib nbextension install --user

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
cd /content/drive/MyDrive/python packages/

In [ ]:
import os
os.chdir('/content/drive/MyDrive/python packages')
#!pip install -r /content/drive/MyDrive/python packages/requirements.txt

In [ ]:
#!pip install xgboost
#!jupyter nbextension enable hinterland/hinterland

In [ ]:
#!pip install scikit-plot

In [ ]:
!pip install langdetect
!pip install langid
#!pip install spacy

# Import the Required Libraries for the Project Task

In [ ]:
from spacy.lang.en import English
import langid
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import ConfusionMatrixDisplay
import spacy
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import seaborn as sns
%matplotlib inline
import string
from langdetect import detect
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#import scikitplot as skplt
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.svm import SVC
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
import xgboost as xgb

In [ ]:
#Download NLTK resources (only needed if not downloaded previously)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the Corpus as Csv

In [ ]:
# Set 'display.max_colwidth' to None to show the full content of all columns
#pd.set_option('display.max_colwidth', None)

# Set 'display.max_rows' to None to show all rows without truncation
#pd.set_option('display.max_rows', None)

In [ ]:
#resume_corpus = pd.read_csv(r"C:\Users\HP\Desktop\dataset\Resume.csv",sep=',')
resume_corpus = pd.read_csv("/content/drive/MyDrive/dataset/Resume.csv",sep=',')

In [ ]:
resume_corpus.head()

In [ ]:
resume_corpus.columns

In [ ]:
resume_corpus.shape

# Class Percentage

In [ ]:
# Compute the class percentage
class_percentage = resume_corpus['Category'].value_counts()/len(resume_corpus)*100
# diplay the result
class_percentage

# Class Frequency

In [ ]:
class_frequency = resume_corpus.Category.value_counts()
class_frequency

# Visualise the Class Frequency

In [ ]:
class_percentage.plot(kind = 'bar',rot=85)

# Visualise the Class Distribution

In [ ]:
# Plotting the value counts of the 'Category' column as a bar plot
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(x='Category', data=resume_corpus)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Number of Resumes per Category')
plt.xticks(rotation=85)
plt.show()

##  Visualise the Class Frequency and Percentage

In [ ]:
# Calculate class frequencies
class_freq =  resume_corpus['Category'].value_counts()
class_percentage = class_freq / len(resume_corpus) * 100

# Plot the class frequency with percentage using seaborn barplot
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=class_freq.index, y=class_freq.values)
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Class Frequency')
plt.ylim(top=max(class_freq.values) * 1.1)

# Rotate x-axis tick labels by 70 degrees
ax.set_xticklabels(ax.get_xticklabels(), rotation=70)

# Add percentage labels on the bars
for i, v in enumerate(class_percentage.values):
    plt.text(i, v + 10, f'{v:.1f}%', ha='center',fontsize=6,color='white')

In [ ]:
category_counts = resume_corpus['Category'].value_counts()
total_resumes = len(resume_corpus)
category_percentages = (category_counts / total_resumes) * 100

# Plotting the value counts of the 'Category' column as a count plot
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='Category', data=resume_corpus)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Number and Percentage of Resumes per Category\n')

# Add percentage values on top of the bars
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height + 5, f'{height}\n({height/total_resumes*100:.1f}%)',
            ha='center', fontsize=6, color='black')

plt.xticks(rotation=85)
plt.show()

# Check for Missing or Null Values

In [ ]:
resume_corpus.isna().sum()

## View How the class are distributed

In [ ]:

# Plot the scatterplot with the y-axis (count) on the x-axis and the x-axis labels on the y-axis
sns.scatterplot(x=category_counts, y=resume_corpus['Category'].unique())

# Add labels and title
plt.xlabel('Count')
plt.ylabel('Categories')
plt.title('Category Counts Scatter Plot')

# Show the plot
plt.show()

# Examine the information about the corpus

In [ ]:
resume_corpus.info()

In [ ]:
resume_corpus.Category.nunique()

In [ ]:

# View number of unique words
resume_corpus.nunique()

In [ ]:
resume_corpus.duplicated().sum()

In [ ]:
# View unique strings
resume_corpus.Resume_str.unique()

# Feature Engineering and Exploratory Data Analysis

## Identify Languages in each row of Resume the Resume Column

In [ ]:

# Step 1: Language Identification
def identify_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

resume_corpus['Language'] = resume_corpus['Resume_str'].apply(identify_language)

resume_corpus

## Identify non-english language in the resume rows

In [ ]:
resume_corpus[resume_corpus['Language']!='en']

## Retrieve the Length and Number of Puctuation in each row in the Resume column

In [ ]:
#resume_corpus = pd.DataFrame(data)

# Step 1: Create a function to count punctuation characters and length
def count_punctuations_and_length(text):
    num_punctuations = sum(1 for char in text if char in string.punctuation)
    text_length = len(text)
    return pd.Series([num_punctuations, text_length])

# Step 2: Apply the function to the 'Resume_str' column and assign to new columns
resume_corpus[['Punctuation_Count', 'Text_Length']] = resume_corpus['Resume_str'].apply(count_punctuations_and_length)
# Step 3: Display the updated DataFrame
display(resume_corpus.iloc[:10][:8])

## Count the number of words in each resume rows

In [ ]:
# Step 1: Create a CountVectorizer to calculate word frequencies
count_vectorizer = CountVectorizer()

# Step 2: Fit and transform the 'Resume_str' column to get word frequencies
word_frequencies = count_vectorizer.fit_transform(resume_corpus['Resume_str'])

# Step 3: Get the feature names (words) from the CountVectorizer
feature_names = count_vectorizer.get_feature_names_out()

# Step 4: Create a DataFrame for word frequencies and add to the original DataFrame
word_freq_df = pd.DataFrame(word_frequencies.toarray(), columns=[f"{word}_freq" for word in feature_names])
resume_corpus = pd.concat([resume_corpus, word_freq_df], axis=1)

# Step 5: Display the updated DataFrame
display(resume_corpus)

## Extract features for machine learning

In [ ]:
resume_data = resume_corpus[['Punctuation_Count' ,'Text_Length']]
resume_data

In [ ]:
resume_class = resume_corpus.Category
resume_class.unique()

In [ ]:
# view features description
resume_data.describe()

In [ ]:
resume_data.isna().sum()

The feature contains extreme values(oultiers), notice the difference between the mean and the max values

# Visualise the data with boxplot

In [ ]:
sns.boxplot(resume_data)

Notice the ouliers expecially in the text_length

## Visualise the feature distribution

In [ ]:
sns.displot(resume_data)

the text_length seems to be almost righly skewed

## Convert the class label to numerical values

In [ ]:
# Create a mapping dictionary for the 'class' column
class_mapping = {category: i for i, category in enumerate(resume_corpus['Category'].unique())}

# Map the 'class' column to numerical values
resume_corpus['label'] = resume_corpus['Category'].map(class_mapping)

# Display the updated DataFrame
display(resume_corpus['label'].head())

# X-Boost Model without applying any vectorization matrix on our test

In [ ]:
# Step 2: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resume_data, resume_corpus['label'], test_size=0.2, random_state=42)

# Step 4: Initialize the XGBoost classifier
xclassifier = xgb.XGBClassifier(
    objective='multi:softmax',  # For multi-class classification
    num_class=24,  # Number of classes
    booster='gbtree',  # Tree-based models
    max_depth=5,  # Maximum depth of the individual trees (you can tune this hyperparameter)
    learning_rate=0.1,  # Learning rate (you can tune this hyperparameter)
    n_estimators=100  # Number of boosting rounds (you can tune this hyperparameter)
)

# Step 5: Train the classifier on the training data
xclassifier.fit(X_train, y_train)

# Step 6: Make predictions on the test data
y_pred = xclassifier.predict(X_test)

# Step 7: Evaluate the classifier's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report for detailed performance metrics
print(classification_report(y_test, y_pred))

Poorly performed model, just 6% accuracy. So let go ahead and do the real data cleaning, text processing, etc.

# Data Cleaning

In [ ]:
# view the columns
resume_corpus.columns

In [ ]:
#  Select the important columns
resume_corpus_refined = resume_corpus[['Resume_str','Category']].copy()
resume_corpus_refined.head()

## Rename the Category to class for clarity

In [ ]:
# Renaming the 'Category' column to 'class'
resume_corpus_refined.rename(columns={'Resume_str': 'resume', 'Category': 'class'}, inplace=True)
#resume_corpus_refined = resume_corpus_refined.rename(columns={'Resume_str': 'resume', 'Category': 'class'})

In [ ]:
resume_corpus_refined.head()

# Convert the class strings to numerical labels

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'class' column to numerical labels
resume_corpus_refined['class_numerical'] = label_encoder.fit_transform(resume_corpus_refined['class'])

# Display the updated DataFrame
display(resume_corpus_refined.head())

In [ ]:
# Get the unique values of the 'class' column
unique_class_values = resume_corpus_refined['class'].unique()

# Get the unique values of the 'class_numerical' column
unique_class_numerical_values = resume_corpus_refined['class_numerical'].unique()

# Create a dictionary matching unique class values with numerical class values
class_mapping_dict = {class_value: class_numerical_value for class_value, class_numerical_value in zip(unique_class_values, unique_class_numerical_values)}

# Print the unique values and their corresponding numerical values together
print("Unique values and their corresponding numerical values:")
for class_value, class_numerical_value in class_mapping_dict.items():
    print(f"'{class_value}' -> {class_numerical_value}")

## Get the Class Labels

In [ ]:
resume_corpus_refined['class'].unique()

# Text Preprocessing

## Method 1: Preprocess Text with nltk

In [ ]:
#nltk.download('omw')

In [ ]:
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [ ]:
# Text preprocessing function
def preprocess_text_nltk(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove special characters and numbers
    tokens = [token for token in tokens if token.isalpha()]
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    # Perform stemming or lemmatization (choose one)
    porter = PorterStemmer()
    tokens = [porter.stem(token) for token in tokens]
    #lemmatizer = WordNetLemmatizer()
    #tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

# Apply preprocessing to 'resume' column
resume_corpus_refined['Cleaned_Resume'] = resume_corpus_refined['resume'].apply(preprocess_text_nltk)

# Now cv_data['Cleaned_Resume'] column contains the preprocessed text data

# Print a sample preprocessed resume
print(resume_corpus_refined['Cleaned_Resume'].iloc[0])

## Vectorize Text with TFIDF

In [ ]:
# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Cleaned_Resume' text data into TF-IDF vectors
tfidf_vectors = tfidf_vectorizer.fit_transform(resume_corpus_refined['Cleaned_Resume'])

# Convert the TF-IDF vectors to a DataFrame for better readability (optional)
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Now you have the preprocessed text data represented as TF-IDF vectors in 'tfidf_vectors'
# The 'tfidf_df' DataFrame contains the same information but in a more readable format

# Print the first few rows of the 'tfidf_df' DataFrame
display(tfidf_df.head())

## Display non-zero vectors

In [ ]:
# Step 1: Calculate the number of non-zero values for each feature
num_non_zero_values = (tfidf_df > 0).sum()

# Step 2: Repeat the feature names based on the number of non-zero values for each feature
feature_names_repeated = tfidf_df.columns.repeat(num_non_zero_values)

# Step 3: Repeat the non-zero values for each feature
non_zero_values_repeated = tfidf_df.values[tfidf_df > 0]

# Step 4: Create a new DataFrame with the non-zero TF-IDF scores and their corresponding feature names
non_zero_tfidf_df = pd.DataFrame({'TF-IDF Score': non_zero_values_repeated}, index=feature_names_repeated)

# Display the DataFrame with non-zero TF-IDF scores and their feature names
display(non_zero_tfidf_df)

# Build the First Baseline Model with our TFIDF Vector

## Model 1: Logistic Regression Classifier

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors, resume_corpus_refined['class_numerical'], test_size=0.2, random_state=0,stratify = resume_corpus_refined['class_numerical'])

# Initialize the logistic regression classifier
classifier = LogisticRegression()

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

## Compute and plot the confusion matrix for the baseline model(Model 1)

In [ ]:
# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels
plt.figure(figsize=(18, 16))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Numbers)')
plt.show()

plt.figure(figsize=(18, 16))
sns.heatmap(cm_percentage, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Percentages)')
plt.show()

## Model 2 :Support Vector Machine Classifier with ITIDF Vectors

In [ ]:

# Initialize the Support Vector Machine (SVM) classifier
# For binary classification, use SVC(kernel='linear')
# For multiclass classification, use SVC(kernel='linear', decision_function_shape='ovr') or SVC(kernel='linear', decision_function_shape='ovo')
svm_classifier = SVC(kernel='linear', decision_function_shape='ovr')

# Train the classifier on the training data
svm_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred_2 = svm_classifier.predict(X_test)
y_pred_2

## Compute and Plot the Confusion Matrix

In [ ]:
# Create a confusion matrix
cm_2 = confusion_matrix(y_test, y_pred_2)

# Calculate the percentages
cm_percentage_2 = cm_2.astype('float') / cm_2.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Numbers)')
plt.show()

plt.figure(figsize=(14, 12))
sns.heatmap(cm_percentage, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Percentages)')
plt.show()

##  Evaluate the Model Performance

In [ ]:
# Evaluate the model's performance
accuracy_2 = accuracy_score(y_test, y_pred_2)
print("Accuracy:", accuracy)

report_2 = classification_report(y_test, y_pred_2)
print("Classification Report:\n", report_2)

## Model 3: Random Forest Model

In [ ]:

# plit the data into training and testing sets
#X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(tfidf_vectors, resume_corpus_refined['class'], test_size=0.2, random_state=42)

# Initialize the Random Forest classifier with different variable names
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest classifier on the training data
random_forest_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = random_forest_classifier.predict(X_test)

# Create a confusion matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)

# Calculate the percentages
cm_percentage_rf = cm_rf.astype('float') / cm_rf.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels_rf = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels for Random Forest model
plt.figure(figsize=(10, 8))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels_rf, yticklabels=class_labels_rf)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Random Forest Confusion Matrix (Numbers)')
plt.show()

plt.figure(figsize=(14, 12))
sns.heatmap(cm_percentage_rf, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels_rf, yticklabels=class_labels_rf)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Random Forest Confusion Matrix (Percentages)')
plt.show()

## Evaluate the Model Performance

In [ ]:
# Evaluate the Random Forest model's performance
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

report_rf = classification_report(y_test, y_pred_rf)
print("Random Forest Classification Report:\n", report_rf)

In [ ]:
#!python -m spacy download en_core_web_lg

# Preprocess Text with Spacy

## Examing the First Resume Content

In [ ]:
nlp = spacy.load("en_core_web_md")
doc = nlp(resume_corpus_refined['resume'][0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_,token.is_oov,token.has_vector,token.is_stop,token.vocab)

In [ ]:
doc.has_vector

## Process the Text

In [ ]:
# Load the English language model for spaCy
#nlp = spacy.load("en_core_web_sm")

# Define a function to preprocess text using spaCy
#def preprocess_text_spacy(text):
    # Convert text to lowercase
 #   text = text.lower()
#
 #   # Remove special characters and punctuation using spaCy tokenizer
  #  doc = nlp(text)
   # words = [token.text for token in doc if token.is_alpha]

    # Remove stopwords using spaCy
    #words = [token.text for token in doc if not token.is_stop]

    # Lemmatize the words using spaCy
    #words = [token.lemma_ for token in doc]

    # Join the processed words back into a single string
    #processed_text = ' '.join(words)

    #return processed_text

# Assuming 'resume_corpus_refined' is a list of resume texts
#preprocessed_resumes_spacy = [preprocess_text_spacy(resume) for resume in resume_corpus_refined['resume']]

In [ ]:
# Load the English language model for spaCy
nlp = spacy.load("en_core_web_md")
# Define a function to preprocess text and obtain word embeddings using spaCy
def preprocess_text_and_get_embeddings(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters and punctuation using spaCy tokenizer
    doc = nlp(text)
    words = [token.text for token in doc if token.is_alpha and not token.is_punct]
    # Remove stopwords using spaCy
    words = [token.text for token in doc if not token.is_stop]

    # Lemmatize the words using spaCy
    words = [token.lemma_ for token in doc]

    # Join the processed words back into a single string
    processed_text = ' '.join(words)

    # Get the word embedding vector for the processed text
    word_embedding_vector = doc.vector

    return processed_text, word_embedding_vector

# Assuming 'resume_corpus_refined' is a DataFrame with a column named 'resume'
# You can replace 'resume_corpus_refined' with your actual DataFrame name
# Create lists to store the processed text and word embedding vectors
processed_texts = []
word_embedding_vectors = []

# Iterate through each resume and preprocess it using spaCy while obtaining word embeddings
for index, row in resume_corpus_refined.iterrows():
    processed_text, word_embedding_vector = preprocess_text_and_get_embeddings(row['resume'])
    processed_texts.append(processed_text)
    word_embedding_vectors.append(word_embedding_vector)

# Create a new DataFrame from the collected lists
processed_resumes_with_embeddings = pd.DataFrame({'processed_text': processed_texts,
                                                  'word_embeddings': word_embedding_vectors})

In [ ]:
processed_resumes_with_embeddings

# Model 3: Logistic Classifier with Spacy MD

In [ ]:
# Split the data into features (word embeddings) and labels (target)
X = processed_resumes_with_embeddings['word_embeddings'].tolist()
y = resume_corpus_refined['class_numerical']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify = resume_corpus_refined['class_numerical'])

# Train the classifier on the scaled data
classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
classifier.fit(X_train, y_train)# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get a detailed classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

## Process text with spacy lg

In [ ]:

# Load the large English language model for spaCy
nlp_large = spacy.load("en_core_web_lg")

# Define a function to preprocess text and obtain word embeddings using spaCy
def preprocess_text_and_get_embeddings_large(text):
    # Convert text to lowercase
    processed_text = text.lower()

    # Remove special characters and punctuation using spaCy tokenizer
    doc = nlp_large(processed_text)
    words = [token.text for token in doc if token.is_alpha and not token.is_punct]

    # Remove stopwords using spaCy
    words = [token.text for token in doc if not token.is_stop]

    # Lemmatize the words using spaCy
    words = [token.lemma_ for token in doc]

    # Join the processed words back into a single string
    processed_text = ' '.join(words)

    # Get the word embedding vector for the processed text
    word_embedding_vector = doc.vector

    return processed_text, word_embedding_vector

# Assuming 'resume_corpus_refined' is a DataFrame with a column named 'resume'
# You can replace 'resume_corpus_refined' with your actual DataFrame name
# Create lists to store the processed text and word embedding vectors
processed_texts_list = []
word_embedding_vectors_list = []

# Iterate through each resume and preprocess it using spaCy while obtaining word embeddings
for index, row in resume_corpus_refined.iterrows():
    processed_text, word_embedding_vector = preprocess_text_and_get_embeddings_large(row['resume'])
    processed_texts_list.append(processed_text)
    word_embedding_vectors_list.append(word_embedding_vector)

# Create a new DataFrame from the collected lists
processed_resumes_with_embeddings_lg = pd.DataFrame({'processed_text': processed_texts_list,
                                                  'word_embeddings': word_embedding_vectors_list}

Model4: spacy large

In [ ]:
# Split the data into features (word embeddings) and labels (target)
X = processed_resumes_with_embeddings_lg['word_embeddings'].tolist()
y = resume_corpus_refined['class_numerical']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify = resume_corpus_refined['class_numerical'])

# Initialize the SVM classifier
classifier = SVC()

# Train the SVM classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)
p
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get a detailed classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
# display preprocessed text in a dataframe
resume_corpus_refined['cleaned_resume_spacy_sm'] = preprocessed_resumes_spacy
resume_corpus_refined

In [ ]:
# Define a function to obtain word embeddings for a given resume text
def get_word_embeddings(text):
    doc = nlp(text)
    return doc.vector

# Apply the function to get word embeddings for all resume texts
resume_corpus_refined['word_embeddings'] = resume_corpus_refined['cleaned_resume_spacy_sm'].apply(get_word_embeddings)

# Convert the word embeddings into separate columns in the DataFrame
word_embeddings_df = pd.DataFrame(resume_corpus_refined['word_embeddings'].to_list())

# Concatenate the original DataFrame with the word embeddings DataFrame
resume_corpus_with_word_embeddings = pd.concat([resume_corpus_refined, word_embeddings_df], axis=1)

In [ ]:

    # embedding vector
    resume_corpus_with_word_embeddings

In [ ]:
# Assuming 'resume_corpus_refined' is a dictionary with 'cleaned_resume' as the key
resumes = resume_corpus_refined['cleaned_resume_spacy_sm']

# Create an empty list to store the vectors for each resume
resume_vectors = []

# Loop through each resume text and vectorize it using spaCy
for resume_text in resumes:
    doc = nlp(resume_text)
    resume_vectors.append(doc.vector)

In [ ]:
# Capture out-of-vocabulary (OOV) words
oov_words = set()
doc = nlp(resume_corpus_refined['cleaned_resume_spacy_sm'].to_string())
for token in doc:
    if not token.has_vector:  # Check if the word has a vector representation (i.e., recognized by the model)
        oov_words.add(token.text)

print("Out-of-Vocabulary (OOV) Words:", oov_words)

In [ ]:

# Assuming 'resume_corpus_refined' is a list of cleaned resume texts as strings
# Process each resume text to create spaCy Doc objects
resumes = [nlp(resume_text) for resume_text in resume_corpus_refined['cleaned_resume_spacy_sm']]

# Get individual sentences from each resume
sentences = [list(resume.sents) for resume in resume_corpus_refined['cleaned_resume_spacy_sm']].to_list()

# Flatten the list of sentences to form a list of words
words = [str(word) for sentence in sentences for word in sentence]

# Train Word2Vec model
embedding_size = 100  # You can adjust this to the desired embedding size
model = Word2Vec(sentences=words, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Save the model or use it for further analysis
model.save("word2vec_model.bin")

In [ ]:
resume_corpus_refined['cleaned_resume_spacy_sm'].to_list()

In [ ]:
model

In [ ]:

# Load the trained Word2Vec model
model = Word2Vec.load("word2vec_model.bin")

# Get word vectors for some sample words
#sample_words = ["resume", "skills", "experience", "job", "education"]
word_vectors = [model.wv[word] for word in resume_corpus_refined['cleaned_resume_spacy_sm']]

# Find most similar words to a given word
similar_words = model.wv.most_similar(resume_corpus_refined['cleaned_resume_spacy_sm'])

# Example sentences]
example_sentences = [
    "I have strong communication skills and extensive experience in project management.",
    "My job responsibilities include analyzing data and creating insightful reports.",
    "Education plays a crucial role in shaping one's career.",
    "The candidate's resume was impressive, showcasing relevant skills and achievements.",
]

print("Sample Word Vectors:")
for word, vector in zip(sample_words, word_vectors):
    print(f"{word}: {vector}")

print("\nMost Similar Words to 'resume':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

print("\nExample Sentences:")
for sentence in example_sentences:
    print(sentence)

In [ ]:

# Assuming 'resume_corpus_refined' is a list of cleaned resume texts as strings
# Process each resume text to create spaCy Doc objects
resumes = [nlp(resume_text) for resume_text in resume_corpus_refined['cleaned_resume_spacy_sm']]

# Get individual sentences from each resume
sentences = [list(resume.sents) for resume in resumes]

# Flatten the list of sentences to form a list of words
words = [str(word) for sentence in sentences for word in sentence]

# Train Word2Vec model
embedding_size = 100  # You can adjust this to the desired embedding size
model = Word2Vec(sentences=words, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Save the model or use it for further analysis
model.save("word2vec_model.bin")

In [ ]:
from gensim.models import Word2Vec

In [ ]:
cd C:\Users\HP\Desktop\Msc Reinforcement learning\

In [ ]:
path = r"C:\Users\HP\Desktop\Msc Reinforcement learning\en_core_web_lg-3.6.0-py3-none-any.whl"

In [ ]:
import wheel
import zipfile
import os

# Replace "path/to/en_core_web_lg-3.6.0-py3-none-any.whl" with the actual path to your wheel file
wheel_file_path = r"C:\Users\HP\Desktop\Msc Reinforcement learning\en_core_web_lg-3.6.0-py3-none-any.whl"

# Create a directory to extract the contents of the wheel file
extracted_dir = r"C:\Users\HP\Desktop\Msc Reinforcement learning"
os.makedirs(extracted_dir, exist_ok=True)

# Extract the wheel file
with zipfile.ZipFile(wheel_file_path, "r") as zip_ref:
    zip_ref.extractall(extracted_dir)

In [ ]:
path = r"C:\Users\HP\Desktop\Msc Reinforcement learning\en_core_web_lg\en_core_web_lg-3.6.0"

In [ ]:
nlp_lg = spacy.load(path)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the spaCy English language model "en_core_web_lg"
#nlp_lg = spacy.load(r"C:\Users\HP\Desktop\Msc Reinforcement learning\en_core_web_lg-3.6.0-py3-none-any.whl")

# Assuming you have already loaded the data into a DataFrame named resume_corpus_refined
# resume_corpus_refined = pd.read_csv("path_to_your_csv_file.csv")

# Text preprocessing function using spaCy with the "en_core_web_lg" model
def preprocess_text_spacy_lg(text):
    doc = nlp_lg(text)
    # Your preprocessing code using spaCy with "en_core_web_lg" (e.g., lemmatization, removing stop words, etc.)
    processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return processed_text

# Apply preprocessing using spaCy to 'Resume_str' column if not done earlier
resume_corpus_refined['Cleaned_Resume_lg'] = resume_corpus_refined['Resume'].apply(preprocess_text_spacy_lg)

# Initialize the TF-IDF Vectorizer for the "en_core_web_lg" processed text
embedding_vectorizer_lg = [token.vector for token in doc]

# Convert the list of embeddings to a numpy array
embedding_vectors_lg = np.array(embedding_vectorizer_lg)
# Split the data into training and testing sets
X_train_lg, X_test_lg, y_train_lg, y_test_lg = train_test_split(embedding_vectors_lg, resume_corpus_refined['class'], test_size=0.2, random_state=42)

# Initialize the Multinomial Naive Bayes classifier for "en_core_web_lg" processed text with different variable names
naive_bayes_classifier_lg = MultinomialNB()

# Train the Multinomial Naive Bayes classifier on the training data
naive_bayes_classifier_lg.fit(X_train_lg, y_train_lg)

# Make predictions on the test data
y_pred_lg = naive_bayes_classifier_lg.predict(X_test_lg)

y_pred_lg[:10]

In [ ]:
# Text preprocessing function using spaCy with the "en_core_web_lg" model
def preprocess_text_spacy_lg(text):
    doc = nlp_lg(text)
    # Your preprocessing code using spaCy with "en_core_web_lg" (e.g., lemmatization, removing stop words, etc.)
    processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return processed_text

# Apply preprocessing using spaCy to 'Resume_str' column if not done earlier
resume_corpus_refined['Cleaned_Resume_lg'] = resume_corpus_refined['Resume'].apply(preprocess_text_spacy_lg)

# Create a vector representation for each document using the embeddings of "en_core_web_lg"
embedding_vectors_lg = np.array([doc.vector for doc in nlp_lg.pipe(resume_corpus_refined['Cleaned_Resume_lg'])])

# Split the data into training and testing sets
X_train_lg, X_test_lg, y_train_lg, y_test_lg = train_test_split(embedding_vectors_lg, resume_corpus_refined['Category'], test_size=0.2, random_state=42)

# Initialize the Multinomial Naive Bayes classifier for "en_core_web_lg" processed text with different variable names
naive_bayes_classifier_lg = MultinomialNB()

# Train the Multinomial Naive Bayes classifier on the training data
naive_bayes_classifier_lg.fit(X_train_lg, y_train_lg)

# Make predictions on the test data
y_pred_lg = naive_bayes_classifier_lg.predict(X_test_lg)

# View the first 10 predictions
print(y_pred_lg[:10])

# Plot the Confusion Matrix

In [ ]:
# Create a confusion matrix
cm_lg = confusion_matrix(y_test_lg, y_pred_lg)

# Calculate the percentages
cm_percentage_lg = cm_lg.astype('float') / cm_lg.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels_lg = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels for the Multinomial Naive Bayes model with "en_core_web_lg"
plt.figure(figsize=(10, 8))
sns.heatmap(cm_lg, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels_lg, yticklabels=class_labels_lg)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Multinomial Naive Bayes Confusion Matrix with en_core_web_lg (Numbers)')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage_lg, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels_lg, yticklabels=class_labels_lg)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Multinomial Naive Bayes Confusion Matrix with en_core_web_lg (Percentages)')
plt.show()

# Evaluate the Model Performance

In [ ]:

# Evaluate the Multinomial Naive Bayes model's performance with "en_core_web_lg" processed text
accuracy_lg = accuracy_score(y_test_lg, y_pred_lg)
print("Multinomial Naive Bayes Accuracy with en_core_web_lg:", accuracy_lg)

report_lg = classification_report(y_test_lg, y_pred_lg)
print("Multinomial Naive Bayes Classification Report with en_core_web_lg:\n", report_lg)

In [ ]:

# Apply preprocessing using spaCy to 'Resume_str' column if not done earlier
resume_corpus_refined['cleaned_Resume'] = resume_corpus_refined['resume'].apply(preprocess_text_spacy)

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer_spacy = TfidfVectorizer()

# Fit and transform the 'Cleaned_Resume' text data into TF-IDF vectors
tfidf_vectors_spacy = tfidf_vectorizer_spacy.fit_transform(resume_corpus_refined['cleaned_resume'])

# Split the data into training and testing sets
X_train_spacy, X_test_spacy, y_train_spacy, y_test_spacy = train_test_split(tfidf_vectors_spacy, resume_corpus_refined['class'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression classifier with different variable names
logistic_classifier_spacy = LogisticRegression(max_iter=1000, random_state=42)

# Train the Logistic Regression classifier on the training data
logistic_classifier_spacy.fit(X_train_spacy, y_train_spacy)

# Make predictions on the test data
y_pred_spacy = logistic_classifier_spacy.predict(X_test_spacy)

# Create a confusion matrix
cm_spacy = confusion_matrix(y_test_spacy, y_pred_spacy)

# Calculate the percentages
cm_percentage_spacy = cm_spacy.astype('float') / cm_spacy.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels_spacy = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels for the Logistic Regression model with spaCy
plt.figure(figsize=(10, 8))
sns.heatmap(cm_spacy, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels_spacy, yticklabels=class_labels_spacy)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Logistic Regression Confusion Matrix with spaCy (Numbers)')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage_spacy, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels_spacy, yticklabels=class_labels_spacy)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Logistic Regression Confusion Matrix with spaCy (Percentages)')
plt.show()

# Evaluate the Logistic Regression model's performance with spaCy
accuracy_spacy = accuracy_score(y_test_spacy, y_pred_spacy)
print("Logistic Regression Accuracy with spaCy:", accuracy_spacy)

report_spacy = classification_report(y_test_spacy, y_pred_spacy)
print("Logistic Regression Classification Report with spaCy:\n", report_spacy)

# Apply preprocessing using spaCy to 'Resume_str' column if not done earlier
resume_corpus_refined['cleaned_resume'] = resume_corpus_refined['Resume'].apply(preprocess_text_spacy)

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer_spacy = TfidfVectorizer()

# Fit and transform the 'Cleaned_Resume' text data into TF-IDF vectors
tfidf_vectors_spacy = tfidf_vectorizer_spacy.fit_transform(resume_corpus_refined['cleaned_resume'])

# Split the data into training and testing sets
X_train_spacy, X_test_spacy, y_train_spacy, y_test_spacy = train_test_split(tfidf_vectors_spacy, resume_corpus_refined['class'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression classifier with different variable names
logistic_classifier_spacy = LogisticRegression(random_state=42)

# Train the Logistic Regression classifier on the training data
logistic_classifier_spacy.fit(X_train_spacy, y_train_spacy)

# Make predictions on the test data
y_pred_spacy = logistic_classifier_spacy.predict(X_test_spacy)

# Create a confusion matrix
cm_spacy = confusion_matrix(y_test_spacy, y_pred_spacy)

# Calculate the percentages
cm_percentage_spacy = cm_spacy.astype('float') / cm_spacy.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels_spacy = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels for the Logistic Regression model with spaCy
plt.figure(figsize=(10, 8))
sns.heatmap(cm_spacy, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels_spacy, yticklabels=class_labels_spacy)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Logistic Regression Confusion Matrix with spaCy (Numbers)')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage_spacy, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels_spacy, yticklabels=class_labels_spacy)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Logistic Regression Confusion Matrix with spaCy (Percentages)')
plt.show()



## Evaluate the Model Performance

In [ ]:
# Evaluate the Logistic Regression model's performance with spaCy
accuracy_spacy = accuracy_score(y_test_spacy, y_pred_spacy)
print("Logistic Regression Accuracy with spaCy:", accuracy_spacy)

report_spacy = classification_report(y_test_spacy, y_pred_spacy)
print("Logistic Regression Classification Report with spaCy:\n", report_spacy)

In [ ]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the spaCy English language model "en_core_web_lg"
nlp_lg = spacy.load("C:\Users\HP\Desktop\Msc Reinforcement learning\en_core_web_lg-3.6.0-py3-none-any.whl")

# Assuming you have already loaded the data into a DataFrame named resume_corpus_refined
# resume_corpus_refined = pd.read_csv("path_to_your_csv_file.csv")

# Text preprocessing function using spaCy with the "en_core_web_lg" model
def preprocess_text_spacy_lg(text):
    doc = nlp_lg(text)
    # Your preprocessing code using spaCy with "en_core_web_lg" (e.g., lemmatization, removing stop words, etc.)
    processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return processed_text

# Apply preprocessing using spaCy to 'Resume_str' column if not done earlier
resume_corpus_refined['Cleaned_Resume_lg'] = resume_corpus_refined['Resume'].apply(preprocess_text_spacy_lg)

# Initialize the TF-IDF Vectorizer for the "en_core_web_lg" processed text
tfidf_vectorizer_lg = TfidfVectorizer()

# Fit and transform the 'Cleaned_Resume_lg' text data into TF-IDF vectors
tfidf_vectors_lg = tfidf_vectorizer_lg.fit_transform(resume_corpus_refined['Cleaned_Resume_lg'])

# Split the data into training and testing sets
X_train_lg, X_test_lg, y_train_lg, y_test_lg = train_test_split(tfidf_vectors_lg, resume_corpus_refined['class'], test_size=0.2, random_state=42)

# Initialize the Gradient Boosting classifier for "en_core_web_lg" processed text with different variable names
gradient_boosting_classifier_lg = GradientBoostingClassifier(random_state=42)

# Train the Gradient Boosting classifier on the training data
gradient_boosting_classifier_lg.fit(X_train_lg, y_train_lg)

# Make predictions on the test data
y_pred_lg = gradient_boosting_classifier_lg.predict(X_test_lg)

# Create a confusion matrix
cm_lg = confusion_matrix(y_test_lg, y_pred_lg)

# Calculate the percentages
cm_percentage_lg = cm_lg.astype('float') / cm_lg.sum(axis=1)[:, np.newaxis]

# Get the unique class labels
class_labels_lg = resume_corpus_refined['class'].unique()

# Plot the confusion matrix heatmap with custom labels for the Gradient Boosting model with "en_core_web_lg"
plt.figure(figsize=(10, 8))
sns.heatmap(cm_lg, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_labels_lg, yticklabels=class_labels_lg)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Gradient Boosting Confusion Matrix with en_core_web_lg (Numbers)')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage_lg, annot=True, cmap='Blues', cbar=False, fmt=".2f", xticklabels=class_labels_lg, yticklabels=class_labels_lg)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Gradient Boosting Confusion Matrix with en_core_web_lg (Percentages)')
plt.show()

# Evaluate the Gradient Boosting model's performance with "en_core_web_lg" processed text
accuracy_lg = accuracy_score(y_test_lg, y_pred_lg)
print("Gradient Boosting Accuracy with en_core_web_lg:", accuracy_lg)

report_lg = classification_report(y_test_lg, y_pred_lg)
print("Gradient Boosting Classification Report with en_core_web_lg:\n", report_lg)